In [14]:
# Импорт библиотеки для работы с json-файлами
import json

# импортируем библиотеку numpy и pandas
import pandas as pd
import numpy as np

# импортируем библиотеку для работы с картами
from keplergl import KeplerGl

# импортируем библиотеку для вычисление высоты
import srtm

CGN Airport elevation (meters): 423


In [35]:
# Координаты: [ [ [ 52.4894, 103.878 ], [ 52.0140, 103.900 ], [ 52.0343, 104.724 ], [ 52.5044, 104.707 ], [ 52.4894, 103.878 ] ] ]

In [17]:
elevation_data = srtm.get_data()
print('CGN Airport elevation (meters):', elevation_data.get_elevation(52.274269, 104.274618))

<img src="https://ночная.москва/wp-content/uploads/2021/03/geograficheskie-koordinaty-moskvy-rossiya.png"
     width="100%" height="100%">

In [56]:
up_left = [52.446093, 103.964495]
up_right = [52.446093, 104.802521]

down_left = [52.013882, 103.964495]
down_right = [52.013882, 104.802521]

In [59]:
coordinates = []

coordinates.append(up_left)
coordinates.append(up_right)
coordinates.append(down_left)
coordinates.append(down_right)

headers = ['latitude', 'longitude']

df = pd.DataFrame(coordinates, columns = headers)
df

,latitude,longitude
0,52.446093,103.964495
1,52.446093,104.802521
2,52.013882,103.964495
3,52.013882,104.802521


In [60]:
with open('config.json') as f:
    config = json.load(f)

map_1 = KeplerGl()
map_1.config = config

map_1.add_data(data=df, name='9673oreu')

map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'prj4oj', 'type': '…